# Setup

In [1]:
# Common imports
import sys
import os
import sklearn
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib.pyplot as plt

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Load the data

In [2]:
# Load datasets from keras.io
tf.keras.datasets.cifar10.load_data()
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
# Split the full training set into a validation set and a training set,
# and scale the pixel intensities down to the 0-1 range and convert them to floats, by dividing by 255.
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

## Create a model using the Sequential API

In [4]:
model = keras.models.Sequential()
# Input layer:
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))

# Hidden layers:
for layer in range(3):
    model.add(keras.layers.Dense(80, activation="relu"))

model.add(keras.layers.AlphaDropout(rate=0.10))
# Output layer.
model.add(keras.layers.Dense(10, activation="softmax"))

## Compile the model

In [5]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

## Train the model

In [6]:
# EarlyStopping (with rollback to the best model).
early_stopping_cb = keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)


# Performance scheduling
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=4)

In [7]:
# Train the model with callbacks
history = model.fit(X_train, y_train, epochs=40,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping_cb, lr_scheduler])

Epoch 1/40
1407/1407 [==============================] - 4s 2ms/step - loss: 2.1635 - accuracy: 0.2110 - val_loss: 1.8910 - val_accuracy: 0.3176 - lr: 0.0100
Epoch 2/40
1407/1407 [==============================] - 3s 2ms/step - loss: 1.9323 - accuracy: 0.2990 - val_loss: 1.8437 - val_accuracy: 0.3294 - lr: 0.0100
Epoch 3/40
1407/1407 [==============================] - 3s 2ms/step - loss: 1.8354 - accuracy: 0.3341 - val_loss: 1.8731 - val_accuracy: 0.3496 - lr: 0.0100
Epoch 4/40
1407/1407 [==============================] - 3s 2ms/step - loss: 1.7774 - accuracy: 0.3578 - val_loss: 1.9167 - val_accuracy: 0.3392 - lr: 0.0100
Epoch 5/40
1407/1407 [==============================] - 3s 2ms/step - loss: 1.7408 - accuracy: 0.3713 - val_loss: 1.7404 - val_accuracy: 0.3722 - lr: 0.0100
Epoch 6/40
1407/1407 [==============================] - 4s 3ms/step - loss: 1.7104 - accuracy: 0.3847 - val_loss: 1.7130 - val_accuracy: 0.3834 - lr: 0.0100
Epoch 7/40
1407/1407 [==============================] - 3s

## Evaluate the model

In [8]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.4305 - accuracy: 0.4928


[1.430540680885315, 0.4927999973297119]